In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


In [2]:
# from google.colab import drive
# drive.mount('/content/drive')

In [3]:
# path_to_text = '/content/drive/MyDrive/datasets-spam.csv'
#path_to_text = 'datasets-spam.csv'
path_to_text = 'books_and_authors'
data = pd.read_csv(path_to_text, names=['v1', 'v2'])

# Creating the feature set and label set
label = data['v1']
text = data['v2']

text

0       Robert Cohn was a member, through his father, ...
1       The divorce was arranged and Robert Cohn went ...
2       By that time, though, he had other things to w...
3       The lady who had him, her name was Frances, fo...
4       I first became aware of his lady’s attitude to...
                              ...                        
1068    I was on the point of asking him what that wor...
1069      1. Knowledge of Literature.--Nil.\n  2.     ...
1070    I see that I have alluded above to his powers ...
1071    During the first week or so we had no callers,...
1072    It was upon the 4th of March, as I have good r...
Name: v2, Length: 1073, dtype: object

Прво целосно прочистен

In [4]:
import nltk

nltk.download('omw-1.4')
nltk.download('punkt')

from nltk.corpus import stopwords, wordnet

nltk.download('stopwords')
stopwords_ = stopwords.words('english')

nltk.download('wordnet')

from nltk.stem import WordNetLemmatizer

wnl = WordNetLemmatizer()

nltk.download('averaged_perceptron_tagger')

def get_wordnet_pos(tag):
    if tag.startswith('J'):
        return wordnet.ADJ
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN
import string

[nltk_data] Downloading package omw-1.4 to /home/kicho/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package punkt to /home/kicho/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /home/kicho/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /home/kicho/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/kicho/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [5]:
text_preprocessed = []
for sentence in text:
    #sentence lower
    sentence = sentence.lower()

    #string punct
    sentence = sentence.translate(str.maketrans('', '', string.punctuation))
    #tokenize
    tokens = nltk.word_tokenize(sentence)
    
    # stop-words
    tokens_stop_words = []
    
    for token in tokens:
          if token not in stopwords_:
                tokens_stop_words.append(token)
                
    # Lemmatization
    tokens_lemma = []
    for token in tokens_stop_words:
          tokens_lemma.append(wnl.lemmatize(token, get_wordnet_pos(nltk.pos_tag([token])[0][1])))
            
    final = ' '.join(tokens_lemma)
    
    text_preprocessed.append(final)

Без процесирање, само мали букви и токенизација.

In [6]:
text_preprocessed_1 = []
for sentence in text:
    #sentence lower
    sentence = sentence.lower()
    #tokenize
    tokens = nltk.word_tokenize(sentence)
    
    text_preprocessed_1.append(' '.join(tokens))

In [7]:
len(text_preprocessed_1),len(text_preprocessed), len(text)

(1073, 1073, 1073)

Count Vecotrizer model

In [8]:
train_ratio = .7
train_index= int(len(text)*train_ratio)

trainX, testX = text_preprocessed[:train_index], text_preprocessed[train_index:]
trainY, testY = label[:train_index], label[train_index:]

len(trainX), len(testX) ,len(trainY), len(testY)

(751, 322, 751, 322)

In [9]:
#encode lables (0-ham, 1-spam)
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
trainY= le.fit_transform(trainY)
testY = le.fit_transform(testY)
trainY.shape,testY.shape
trainY[0]

3

In [10]:
from sklearn.feature_extraction.text import CountVectorizer

# Count Vectors as features
# create a count vectorizer object 
count_vect = CountVectorizer(max_features=5000)
count_vect.fit(text_preprocessed)

# transform the training and test data using count vectorizer object
trainX_vec = count_vect.transform(trainX)
testX_vec = count_vect.transform(testX)
trainX_vec.shape,
testX_vec.shape

(322, 5000)

In [11]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, f1_score


#create an instance of the model
lr_model = LogisticRegression(random_state=7)
#train the model
lr_model.fit(trainX_vec, trainY)

#predict test data
pred_test = lr_model.predict(testX_vec)

#print evaluation metrics 
print(classification_report(testY,pred_test))
print(confusion_matrix(testY,pred_test))
print("Accuracy:",accuracy_score(testY, pred_test))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00        98
           1       0.78      0.26      0.39       224
           2       0.00      0.00      0.00         0
           3       0.00      0.00      0.00         0
           4       0.00      0.00      0.00         0
           5       0.00      0.00      0.00         0
           6       0.00      0.00      0.00         0
           7       0.00      0.00      0.00         0

    accuracy                           0.18       322
   macro avg       0.10      0.03      0.05       322
weighted avg       0.55      0.18      0.27       322

[[ 0 16 33  1  0  9 35  4]
 [ 4 58 32  1 10  8 65 46]
 [ 0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0]]
Accuracy: 0.18012422360248448


/home/kicho/anaconda3/envs/DataScience/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/kicho/anaconda3/envs/DataScience/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/kicho/anaconda3/envs/DataScience/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Вториот модел

In [12]:
train_1_ratio = .7
train_1_index= int(len(text)*train_1_ratio)

trainX_1, testX_1 =text_preprocessed_1[:train_index], text_preprocessed_1[train_index:]
trainY_1, testY_1 =label[:train_1_index], label[train_1_index:]

len(trainX_1), len(testX_1) ,len(trainY_1), len(testY_1)

(751, 322, 751, 322)

In [13]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
trainY_1= le.fit_transform(trainY_1)
testY_1 = le.fit_transform(testY_1)
trainY_1.shape,testY_1.shape
trainY_1[0]

3

In [14]:
count_vect_1 = CountVectorizer(max_features=5000)
count_vect_1.fit(text_preprocessed_1)

# transform the training and test data using count vectorizer object
trainX_1_vec = count_vect.transform(trainX_1)
testX_1_vec = count_vect.transform(testX_1)
trainX_1_vec.shape,
testX_1_vec.shape

(322, 5000)

In [15]:
lr_model_1 = LogisticRegression(random_state=0)
lr_model_1.fit(trainX_1_vec, trainY_1)

pred_test_1 = lr_model_1.predict(testX_1_vec)

#print evaluation metrics 
print(classification_report(testY_1,pred_test_1))
print(confusion_matrix(testY_1,pred_test_1))
print("Accuracy:",accuracy_score(testY_1, pred_test_1))

              precision    recall  f1-score   support

           0       0.17      0.05      0.08        98
           1       0.81      0.29      0.42       224
           2       0.00      0.00      0.00         0
           4       0.00      0.00      0.00         0
           5       0.00      0.00      0.00         0
           6       0.00      0.00      0.00         0
           7       0.00      0.00      0.00         0

    accuracy                           0.21       322
   macro avg       0.14      0.05      0.07       322
weighted avg       0.61      0.21      0.32       322

[[ 5 15 46  0  8 20  4]
 [25 64 35 16 15 38 31]
 [ 0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0]]
Accuracy: 0.21428571428571427


/home/kicho/anaconda3/envs/DataScience/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/home/kicho/anaconda3/envs/DataScience/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/kicho/anaconda3/envs/DataScience/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning:

In [16]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC


In [17]:
tfidf = TfidfVectorizer(max_features=5000, stop_words={‘english’}, ngram_range=(1, 2)) 
tfidf.fit(text)

X_train_tfidf = tfidf.transform(trainX)
X_test_tfidf = tfidf.transform(testX)

X_train_tfidf.shape, X_test_tfidf.shape

SyntaxError: invalid character '‘' (U+2018) (2586729214.py, line 1)

In [ ]:
lr_model_tf = LogisticRegression(random_state=0)
lr_model_tf.fit(X_train_tfidf, trainY)
pred_test_tf = lr_model_tf.predict(X_test_tfidf)

#print evaluation metrics 
print(classification_report(testY,pred_test_tf))
print(confusion_matrix(testY,pred_test_tf))
print("Accuracy:",accuracy_score(testY, pred_test_tf))

In [ ]:
lr_model_tf = LogisticRegression(random_state=0)
lr_model_tf.fit(X_train_tfidf, trainY)
pred_test_tf = lr_model_tf.predict(X_test_tfidf)

#print evaluation metrics 
print(classification_report(testY,pred_test_tf))
print(confusion_matrix(testY,pred_test_tf))
print("Accuracy:",accuracy_score(testY, pred_test_tf))

In [ ]:
#Language processing
text.concordance("word") - колку пати се појавува зборот


In [ ]:
# mutlinomial NB, Random Forest, SVM(kernel='linear', probability=True) bi gi probala


In [ ]:
# embeddings 
from gensim.models.word2vec import Word2Vec
from gensim.models import Phrases

bigramer = Phrases(sentences)
model = Word2Vec(bigramer[sentences], size=100, window=5, min_count=10, workers=4)

# unload memory
model.init_sims(replace=True) 

# Storing a model
model.save("author")
# new_model = gensim.models.Word2Vec.load('author')

# Switch to KeyedVectors instance  
w2v = {w: vec for w,vec in zip(model.wv.indx2word, model.wv.syn0)}

In [ ]:
model.most_similar()

In [ ]:
class EmbeddingVectorizer(object):
    # If word2vec were passed in during initialization, use those
    def __init__(self, word2vec):
        self.word2vec = word2vec
        self.word2weight = None
        self.dim = 100
    
    # learning word2weight
    def fit(self, X, y):
        vect = TfidfVectorizer(min_df=5, ngram_range=(1,3))
        vect.fit(X)
        max_idf = max(vect.idf_)
        self.word2weight = defaultdict(
        lambda: max_idf, [(w, vect.idf_[i]) for w, i in vect.vocabulary_.items()]
        )
        return self
    
    # Use learned word2weight
    def transform(self, X):
        return np.array([
            np.mean([
                self.word2vec[w]*self.word2weight[w] 
                for w in words if w in self.word2vec] or 
                [np.zeros(self.dim)], axis=0) 
            for words in X
        ])
        

In [ ]:
from keras.preprocessing.text import Tokenizer #similar to the CountVectorizer and TfIDF from sci-kit

#The word embedding layer expects input sequences to be comprised of integers.
# integer encode sequences of words
tokenizer = Tokenizer()

tokenizer.fit_on_texts(text_preprocessed)

sequences = tokenizer.texts_to_sequences(text_preprocessed)

In [ ]:
len(sequences[])

In [ ]:
tokenizer.word_index

In [ ]:
sequences = np.array(sequences)

train_part= int(len(text)*0.7)
train_X, test_X =sequences[:train_part], sequences[train_part:]

train_X.shape,test_X.shape

In [ ]:
from keras.preprocessing.sequence import pad_sequences

max_len = 30
#transforms a list (of length num_samples) of sequences (lists of integers) 
#into a 2D Numpy array of shape (num_samples, num_timesteps) num_timesteps is the maxlen argument.
train_X_pad = pad_sequences(train_X, maxlen=max_len)
test_X_pad = pad_sequences(test_X, maxlen=max_len)
print(train_X_pad.shape)
print(test_X_pad.shape)

In [ ]:
vocab_size = len(tokenizer.word_index) + 1
vocab_size

In [ ]:
# create the model
from keras.models import Model
from keras.layers import Input,Embedding,LSTM,Dense

def generate_model(vocab_size, max_len, embedding_size):

  _input = Input(max_len)

  x = Embedding(input_dim = vocab_size, output_dim = embedding_size) (_input)

  x = LSTM(100)(x)

  output = Dense(1, activation='sigmoid')(x)

  model = Model(inputs= [_input], outputs = [output])

  model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
  return model

In [ ]:
model = generate_model(vocab_size,max_len,embedding_size=50)
model.summary()

In [ ]:
history = model.fit(train_X_pad, trainY, epochs=10, batch_size=60)

In [ ]:
pred_test = model.predict(test_X_pad)
#pred_test = np.argmax(pred_test,axis=1)
pred_test = pred_test.round()

#print evaluation metrics 
print(classification_report(testY,pred_test))
print(confusion_matrix(testY,pred_test))
print("Accuracy:",accuracy_score(testY, pred_test))